# Explore metadata fields in the full archaeology dataset

In [1]:
import pandas as pd

In [1]:
from pymongo import MongoClient

# Replace with your MongoDB connection string
CONNECTION_STRING = "mongodb://127.0.0.1:27018"  # For local MongoDB

# Connect to the database and collection
client = MongoClient(CONNECTION_STRING)
db = client["archaeology_metadata"] # Create new database
collection = db["collection"] # Create new collection

In [2]:
count = collection.count_documents({})
print(f"Total documents: {count}")

Total documents: 158254


In [29]:
def get_count(path):
    pipeline = [
        { "$match": { f"{path}": { "$exists": True, "$ne": None } } },
        { "$count": "total" }]  

    result = list(collection.aggregate(pipeline))
    count = result[0]["total"] if result else 0

    print(f"{path.split('.')[-1]}: {count}")

    return count



In [ ]:
# Get the count of documents with the field "dansSpatialCoverageText"
get_count("ore:describes.dansTemporalSpatial:dansSpatialCoverageText")
get_count("ore:describes.dansTemporalSpatial:dansSpatialPoint")
get_count("ore:describes.dansTemporalSpatial:dansSpatialBox")
get_count("ore:describes.dansTemporalSpatial:dansSpatialCoverageControlled")
get_count("ore:describes.citation:dsDescription")
get_count("ore:describes.ore:aggregates.schema:name")
get_count("ore:describes.dansRelationMetadata:dansCollection")



dansTemporalSpatial:dansSpatialCoverageText: 154396
dansTemporalSpatial:dansSpatialPoint: 56485
dansTemporalSpatial:dansSpatialBox: 4445
dansTemporalSpatial:dansSpatialCoverageControlled: 8274
citation:dsDescription: 158254
schema:name: 158253
dansRelationMetadata:dansCollection: 43907


43907